# Download Data

### Purpose
This jupyter notebook highlights two different methods for accessing and downloading data from Ocean Observatories Initiative Carbon System instruments. The first method utilizes OOI's API to perform M2M (Machine-2-Machine) queries for data from the OOI THREDDS data server. The second method requests data from OOI's DataExplorer ERDDAP server.

#### THREDDs Data
The data served up via OpenDAP on OOI THREDDs servers are the same datasets which can be accessed via OOI's Data Portal at https://ooinet.oceanobservatories.org/. This is the source for accessing realtime or near-realtime data from OOI. 


#### Data Explorer
Data Explorer is the new tool for exploring, discovering, and downloading data from OOI. It can be accessed via the web at https://dataexplorer.oceanobservatories.org/. Data Explorer hosts "gold copy" versions of OOI datasets, with all the relevant data stream merged into a single unified file. These datasets are hosted on the Data Explorer ERDDAP server at  However, Data Explorer currently only from the Data Explorer website, they currently can't be downloaded from the ERDDAP server.

---
## OOINet/THREDDs
First, we are going to access and download data from OOI's Data Portal. Then we will do some dataset reprocessing to make the resulting data easier and more intuitive to work with. This portion of the notebook relies on some community tools which have been developed by OOI's Data Team members which simplify interacting with OOI's API. The two tools are the OOINet tool (https://github.com/reedan88/OOINet) and the Data Explorations Modules (https://github.com/oceanobservatories/ooi-data-explorations).

This notebook provides an example on how to use the OOINet download tool to perform the following functions:
* Search for datasets
* Identify desired reference designator
* Get the associated metadata for a given reference designator
* Request netCDF datasets for a reference designator
* Download the netCDF dataset to your local machine

The key parameters which the OOI API requires is the "reference designator." A reference designator may be thought of as a type of instrument located at a fixed location and depth. 

In [1]:
# Import libraries
import os, shutil, sys, time, re, requests, csv, datetime, pytz
import time
import yaml
import pandas as pd
import numpy as np
import netCDF4 as nc
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import the OOINet M2M tool
sys.path.append("/home/areed/Documents/OOI/reedan88/ooinet/ooinet/")
from m2m import M2M

#### Set OOINet API access
In order access and download data from OOINet, need to have an OOINet api username and access token. Those can be found on your profile after logging in to OOINet. Your username and access token should NOT be stored in this notebook/python script (for security). It should be stored in a yaml file, kept in the same directory, named user_info.yaml.

In [3]:
userinfo = yaml.load(open("../user_info.yaml"), Loader=yaml.SafeLoader)
username = userinfo["apiname"]
token = userinfo["apikey"]

#### Connect to OOINet

In [4]:
OOINet = M2M(username, token)

---
## Search Datasets
First, we can search the available OOI Reference Designators (i.e. "refdes" for short) on the following keys: **array**, **node**, **instrument**. Additionally, can request for "**English_names**", which will return the descriptive name for the associated array, node, and instrument. Below, we will search for the available CTD instruments on the Pioneer Array Central Surface Mooring.

The major caveat with the search is, similar to searching on ERDDAP datasets, the search terms must be partial or full match based on OOI nomenclature. For example, we have to search for "PCO2", "PCO2AA", or the full instrument name "04-PCO2AA" if we are searching for the sea-surface pCO2 sensor. We can't search "pco2", "carbon dioxide" or other instrument terms.

gold_copy = 'http://thredds.dataexplorer.oceanobservatories.org/thredds/catalog/ooigoldcopy/public/'

In [93]:
instruments = OOINet.search_datasets(array="GP02HYPM", English_names=True)
instruments

https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/GP02HYPM


,array,array_name,node,node_name,instrument,instrument_name,refdes,url,deployments
0,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP03,Wire-Following Profiler Lower,05-VEL3DL000,3-D Single Point Velocity Meter,GP02HYPM-WFP03-05-VEL3DL000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
1,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP03,Wire-Following Profiler Lower,04-CTDPFL000,CTD,GP02HYPM-WFP03-04-CTDPFL000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
2,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP03,Wire-Following Profiler Lower,03-DOSTAL000,Dissolved Oxygen,GP02HYPM-WFP03-03-DOSTAL000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
3,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP03,Wire-Following Profiler Lower,01-FLORDL000,2-Wavelength Fluorometer,GP02HYPM-WFP03-01-FLORDL000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
4,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP03,Wire-Following Profiler Lower,00-WFPENG000,Profiler Controller,GP02HYPM-WFP03-00-WFPENG000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
5,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP02,Wire-Following Profiler Upper,05-VEL3DL000,3-D Single Point Velocity Meter,GP02HYPM-WFP02-05-VEL3DL000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
6,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP02,Wire-Following Profiler Upper,04-CTDPFL000,CTD,GP02HYPM-WFP02-04-CTDPFL000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
7,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP02,Wire-Following Profiler Upper,03-DOSTAL000,Dissolved Oxygen,GP02HYPM-WFP02-03-DOSTAL000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
8,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP02,Wire-Following Profiler Upper,01-FLORDL000,2-Wavelength Fluorometer,GP02HYPM-WFP02-01-FLORDL000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
9,GP02HYPM,Global Station Papa Apex Profiler Mooring,WFP02,Wire-Following Profiler Upper,00-WFPENG000,Profiler Controller,GP02HYPM-WFP02-00-WFPENG000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"


From the returned list of available instruments above, we can select a particular instrument using its **reference designator** (refdes for short):

In [91]:
refdes = "GP02HYPM-WFP02-05-VEL3DL000"

---
## Metadata
Next, we can query OOINet for the metadata associated with the selected reference designator. The metadata contains such valuable information such as the available methods and streams (which are required to download the data), the particleKeys (the data variable names), and the associated units. 

In [50]:
metadata = OOINet.get_metadata(refdes)
metadata

,pdId,particleKey,type,shape,units,fillValue,stream,unsigned,method,count,beginTime,endTime,refdes
0,PD5,density,FLOAT,FUNCTION,kg m-3,-9999999,ctdmo_ghqr_instrument_recovered,False,recovered_inst,280625,2013-07-24T06:45:01.000Z,2021-07-27T17:45:01.000Z,GP03FLMB-RIM01-02-CTDMOG060
1,PD7,time,DOUBLE,SCALAR,seconds since 1900-01-01,-9999999,ctdmo_ghqr_instrument_recovered,False,recovered_inst,280625,2013-07-24T06:45:01.000Z,2021-07-27T17:45:01.000Z,GP03FLMB-RIM01-02-CTDMOG060
2,PD10,port_timestamp,DOUBLE,SCALAR,seconds since 1900-01-01,-9999999,ctdmo_ghqr_instrument_recovered,False,recovered_inst,280625,2013-07-24T06:45:01.000Z,2021-07-27T17:45:01.000Z,GP03FLMB-RIM01-02-CTDMOG060
3,PD11,driver_timestamp,DOUBLE,SCALAR,seconds since 1900-01-01,-9999999,ctdmo_ghqr_instrument_recovered,False,recovered_inst,280625,2013-07-24T06:45:01.000Z,2021-07-27T17:45:01.000Z,GP03FLMB-RIM01-02-CTDMOG060
4,PD12,internal_timestamp,DOUBLE,SCALAR,seconds since 1900-01-01,-9999999,ctdmo_ghqr_instrument_recovered,False,recovered_inst,280625,2013-07-24T06:45:01.000Z,2021-07-27T17:45:01.000Z,GP03FLMB-RIM01-02-CTDMOG060
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,PD16,preferred_timestamp,STRING,SCALAR,1,empty,ctdmo_ghqr_sio_offset,False,telemetered,958,2013-07-24T22:07:18.000Z,2020-01-31T20:07:19.000Z,GP03FLMB-RIM01-02-CTDMOG060
87,PD863,ingestion_timestamp,DOUBLE,SCALAR,seconds since 1900-01-01,-9999,ctdmo_ghqr_sio_offset,False,telemetered,958,2013-07-24T22:07:18.000Z,2020-01-31T20:07:19.000Z,GP03FLMB-RIM01-02-CTDMOG060
88,PD1558,inductive_id,UBYTE,SCALAR,1,0,ctdmo_ghqr_sio_offset,True,telemetered,958,2013-07-24T22:07:18.000Z,2020-01-31T20:07:19.000Z,GP03FLMB-RIM01-02-CTDMOG060
89,PD2174,sio_controller_timestamp,UINT,SCALAR,seconds since 1970-01-01,9999999,ctdmo_ghqr_sio_offset,True,telemetered,958,2013-07-24T22:07:18.000Z,2020-01-31T20:07:19.000Z,GP03FLMB-RIM01-02-CTDMOG060


#### Sensor Parameters
Each instrument returns multiple parameters containing a variety of low-level instrument output and metadata. However, we are interested in science-relevant parameters. We can identify the science parameters based on the preload database, which designates the science parameters with a "data level" of L1 or L2. 

Consequently, we will want to filter and group the metadata for a given reference designator to identify the relevant parameters. First, we query the preload database with the relevant metadata for a reference designator. Then, we filter the metadata for the science-relevant data streams based on the preload information. Then, we reduce the results by grouping by the stream parameter to get the stream-by-stream data, which will be useful when requesting data from OOINet for download. 

In [51]:
data_levels = OOINet.get_parameter_data_levels(metadata)
data_levels

{'PD10': None,
 'PD11': None,
 'PD12': None,
 'PD13': 2,
 'PD1558': None,
 'PD16': None,
 'PD193': 0,
 'PD194': 0,
 'PD195': 0,
 'PD196': None,
 'PD198': None,
 'PD2174': None,
 'PD2226': None,
 'PD2926': 1,
 'PD2927': 1,
 'PD2928': 1,
 'PD312': None,
 'PD5': 2,
 'PD7': None,
 'PD7987': 1,
 'PD7988': 1,
 'PD7989': 1,
 'PD8093': 1,
 'PD8103': 1,
 'PD863': None}

Filter the metadata based on the data levels for **L1** & **L2** data

In [52]:
def filter_parameter_ids(pdId, pid_dict):
    data_level = pid_dict.get(pdId)
    if data_level is not None:
        if data_level > 0:
            return True
        else:
            return False
    else:
        return False

In [53]:
mask = metadata["pdId"].apply(lambda x: filter_parameter_ids(x, data_levels))
metadata = metadata[mask]

Groupby based on the reference designator - method - stream to get the unique values for each data stream

In [54]:
metadata = metadata.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
metadata = metadata.reset_index()
metadata = metadata.applymap(lambda x: x[0] if len(x) == 1 else x)
metadata.head()

,refdes,method,stream,pdId,particleKey,type,shape,units,fillValue,unsigned,count,beginTime,endTime
0,GP03FLMB-RIM01-02-CTDMOG060,bad_telemetered,ctdmo_ghqr_sio_mule_instrument,"[PD5, PD13, PD2926, PD2927, PD2928, PD8093]","[density, practical_salinity, ctdmo_seawater_p...",FLOAT,FUNCTION,"[kg m-3, 1, dbar, ºC, S m-1, m]","[-9999999, -9999]",False,4,2013-07-24T06:32:00.000Z,2022-05-26T21:03:44.219Z
1,GP03FLMB-RIM01-02-CTDMOG060,recovered_host,ctdmo_ghqr_sio_mule_instrument,"[PD5, PD13, PD2926, PD2927, PD2928, PD8093]","[density, practical_salinity, ctdmo_seawater_p...",FLOAT,FUNCTION,"[kg m-3, 1, dbar, ºC, S m-1, m]","[-9999999, -9999]",False,263259,2013-07-24T22:00:01.000Z,2021-07-26T20:00:01.000Z
2,GP03FLMB-RIM01-02-CTDMOG060,recovered_inst,ctdmo_ghqr_instrument_recovered,"[PD5, PD13, PD7987, PD7988, PD7989, PD8103]","[density, practical_salinity, ctdmo_seawater_p...",FLOAT,FUNCTION,"[kg m-3, 1, dbar, ºC, S m-1, m]","[-9999999, -9999]",False,280625,2013-07-24T06:45:01.000Z,2021-07-27T17:45:01.000Z
3,GP03FLMB-RIM01-02-CTDMOG060,telemetered,ctdmo_ghqr_sio_mule_instrument,"[PD5, PD13, PD2926, PD2927, PD2928, PD8093]","[density, practical_salinity, ctdmo_seawater_p...",FLOAT,FUNCTION,"[kg m-3, 1, dbar, ºC, S m-1, m]","[-9999999, -9999]",False,7825,2013-07-24T06:32:00.000Z,2020-02-01T08:00:01.000Z


This returns all of the methods and streams which have scientific data. For PCO2W datasets, we want to drop the entries which have "blank" in them.

In [55]:
mask = metadata["stream"].apply(lambda x: False if "blank" in x else True)
metadata = metadata[mask]
metadata

,refdes,method,stream,pdId,particleKey,type,shape,units,fillValue,unsigned,count,beginTime,endTime
0,GP03FLMB-RIM01-02-CTDMOG060,bad_telemetered,ctdmo_ghqr_sio_mule_instrument,"[PD5, PD13, PD2926, PD2927, PD2928, PD8093]","[density, practical_salinity, ctdmo_seawater_p...",FLOAT,FUNCTION,"[kg m-3, 1, dbar, ºC, S m-1, m]","[-9999999, -9999]",False,4,2013-07-24T06:32:00.000Z,2022-05-26T21:03:44.219Z
1,GP03FLMB-RIM01-02-CTDMOG060,recovered_host,ctdmo_ghqr_sio_mule_instrument,"[PD5, PD13, PD2926, PD2927, PD2928, PD8093]","[density, practical_salinity, ctdmo_seawater_p...",FLOAT,FUNCTION,"[kg m-3, 1, dbar, ºC, S m-1, m]","[-9999999, -9999]",False,263259,2013-07-24T22:00:01.000Z,2021-07-26T20:00:01.000Z
2,GP03FLMB-RIM01-02-CTDMOG060,recovered_inst,ctdmo_ghqr_instrument_recovered,"[PD5, PD13, PD7987, PD7988, PD7989, PD8103]","[density, practical_salinity, ctdmo_seawater_p...",FLOAT,FUNCTION,"[kg m-3, 1, dbar, ºC, S m-1, m]","[-9999999, -9999]",False,280625,2013-07-24T06:45:01.000Z,2021-07-27T17:45:01.000Z
3,GP03FLMB-RIM01-02-CTDMOG060,telemetered,ctdmo_ghqr_sio_mule_instrument,"[PD5, PD13, PD2926, PD2927, PD2928, PD8093]","[density, practical_salinity, ctdmo_seawater_p...",FLOAT,FUNCTION,"[kg m-3, 1, dbar, ºC, S m-1, m]","[-9999999, -9999]",False,7825,2013-07-24T06:32:00.000Z,2020-02-01T08:00:01.000Z


---
## Deployment Information
When we searched for datasets, it returned a table which listed the available deployment numbers for each of the datasets. We can get much more detailed information on the deployments for a particular reference designator by requesting the deployment information from OOINet.

In [56]:
deployments = OOINet.get_deployments(refdes=refdes)
deployments

,deploymentNumber,uid,assetId,latitude,longitude,depth,deployStart,deployEnd,deployCruise,recoverCruise
0,9,CGINS-CTDMOG-13643,1710,50.376423,-144.513125,30.0,2022-05-18 19:57:00,None,SKQ202208S,None
0,8,CGINS-CTDMOG-13624,1681,50.331522,-144.480312,50.0,2021-07-24 19:32:00,2022-05-22 16:59:00,SKQ202111S,SKQ202208S
0,7,CGINS-CTDMOG-10218,1930,50.37555,-144.5145,30.0,2019-09-27 21:26:00,2021-07-27 18:00:00,SKQ201920S,SKQ202111S
0,6,CGINS-CTDMOG-10225,1904,50.329537,-144.397928,30.0,2018-07-24 23:01:00,2019-09-27 23:55:00,SR1811,SKQ201920S
0,5,CGINS-CTDMOG-13422,3083,50.37768,-144.514867,30.0,2017-07-14 22:49:00,2018-07-25 15:11:00,SR1710,SR1811
0,4,CGINS-CTDMOG-11638,2587,50.32925,-144.398,30.0,2016-07-04 23:07:00,2017-07-17 15:02:00,RB1605,SR1710
0,3,CGINS-CTDMOG-12638,3264,50.33033,-144.39833,47.0,2015-06-08 21:25:00,2016-07-03 14:37:00,TN323,RB1605
0,2,CGINS-CTDMOG-11646,2552,50.3325,-144.4053,31.0,2014-06-20 03:20:00,2015-06-06 01:40:00,MV1404,TN323
0,1,CGINS-CTDMOG-10255,2317,50.33167,-144.40083,30.0,2013-07-24 06:32:00,2014-06-18 14:00:00,MV1309,MV1404


We'll go ahead and save the deployment data as a csv since it might be useful when working with the data.

In [57]:
deployments.to_csv(f"../data/{refdes}_deployments.csv", index=False)

---
## Vocab Information
Additionally, if we are interested in more detailed information on the location that the reference designator is assigned to, we can request the vocab information for the given reference designator. The vocab information includes some of the "**English_names**" info we requested when searching for datasets, as well as instrument model, manufacturer, and the descriptive names for the reference designator location.

In [92]:
vocab = OOINet.get_vocab(refdes=refdes)
vocab

,@class,vocabId,refdes,instrument,tocL1,tocL2,tocL3,manufacturer,model,mindepth,maxdepth
0,.VocabRecord,1267,GP02HYPM-WFP02-05-VEL3DL000,3-D Single Point Velocity Meter,Global Station Papa,Apex Profiler Mooring,Wire-Following Profiler Upper,Falmouth Scientific,ACM-Plus,310.0,2093.0


---
## Calibration Information
We can also request the calibration information for a given reference designator. Since individual instruments are swapped during each mooring deployment & recovery, the calibration coefficients for a reference designator are different for each deployment. The way OOI operates is that it loads all the available calibration coefficients for a given reference designator. Then, for each deployment, it finds the calibration coefficients with the most recent calibration date which most closely _precedes_ the start of the deployment. The result is a table, sorted by deployment number for a reference designator, with the uid of the specific instrument, its calibration coefficients, when the instrument was calibrated, and the source of the calibration coefficients.

In [18]:
calibrations = OOINet.get_calibrations(refdes, deployments)
calibrations

Fetching calibrations for GP03FLMA-RIM01-02-CTDMOG040:
[########################################] | 100% Completed | 13.2s


,deploymentNumber,uid,calCoef,value,calFile
0,9,CGINS-CTDMOG-11654,CC_i,-3.899014e-04,CGINS-CTDMOG-11654__20211231_Cal_Info.xlsx
1,9,CGINS-CTDMOG-11654,CC_h,1.480802e-01,CGINS-CTDMOG-11654__20211231_Cal_Info.xlsx
2,9,CGINS-CTDMOG-11654,CC_g,-9.813853e-01,CGINS-CTDMOG-11654__20211231_Cal_Info.xlsx
3,9,CGINS-CTDMOG-11654,CC_pa0,1.110909e-01,CGINS-CTDMOG-11654__20211231_Cal_Info.xlsx
4,9,CGINS-CTDMOG-11654,CC_pa2,-1.399899e-11,CGINS-CTDMOG-11654__20211231_Cal_Info.xlsx
...,...,...,...,...,...
211,1,CGINS-CTDMOG-10257,CC_ptempa0,-6.836014e+01,CGINS-CTDMOG-10257__20121207_Cal_Info.xlsx
212,1,CGINS-CTDMOG-10257,CC_cpcor,-9.570000e-08,CGINS-CTDMOG-10257__20121207_Cal_Info.xlsx
213,1,CGINS-CTDMOG-10257,CC_ctcor,3.250000e-06,CGINS-CTDMOG-10257__20121207_Cal_Info.xlsx
214,1,CGINS-CTDMOG-10257,CC_j,5.352847e-05,CGINS-CTDMOG-10257__20121207_Cal_Info.xlsx


It is also possible to request the calibration history for a specific instrument by utilizing the **uid** of the instrument and using the lower-level ```_get_api``` method and ```OOINet.URLS``` attribute to construct your own request.

In [ ]:
# Set up the calibration url and arguments to pass to the request
cal_url = OOINet.URLS["cal"]
uid = "CGINS-PHSENF-P0183" # This is unique to each instrument
params = {
    "uid": uid
}

# Make the request
calInfo = OOINet._get_api(cal_url, params=params)

# Put the data into a pandas dataframe, sorted by calibration date and coefficient name
columns = ["uid", "calCoef", "calDate", "value", "calFile"]
instrumentCals = pd.DataFrame(columns=columns)
for c in calInfo["calibration"]:
    for cc in c["calData"]:
        instrumentCals = instrumentCals.append({
            "uid": cc["assetUid"],
            "calCoef": cc["eventName"],
            "calDate": OOINet._convert_time(cc["eventStartTime"]),
            "value": cc["value"],
            "calFile": cc["dataSource"]
        }, ignore_index=True)
instrumentCals.sort_values(by=["calDate", "calCoef"], inplace=True)
instrumentCals

---
## Download Datasets
The ultimate goal of the queries above were to identify what data streams(s) we are interested in, along with supporting metadata/calibration information, in order to request the to download. Now we want to be able to request those data streams and get the associated netCDF files. This process involves the following steps:
1. Identify the methods and data streams for the selected reference designator
2. Request the THREDDS server url for the data sets
3. Get the catalog of datasets on the THREDDS server
4. Parse the catalog for the desired netCDF files
5. Download the identified netCDF files to a local directory

Below, we script the above steps in order to download all of the available datasets. In the following section we will combine the data delivered via different methods (e.g. telemetered, recovered_host, recovered_inst) to generate a single combined dataset with the most complete data record available.

In [59]:
def clean_catalog(catalog, stream, deployments):
    """Clean up the netCDF catalog of unwanted datasets"""
    # Parse the netCDF datasets to only get those with the datastream in its name
    datasets = []
    for dset in catalog:
        check = dset.split("/")[-1]
        if stream in check:
            datasets.append(dset)
        else:
            pass
    
    # Next, check that the netCDF datasets are not empty by getting the timestamps in the
    # datasets and checking if they are 
    catalog = datasets
    datasets = []
    for dset in catalog:
        # Get the timestamps
        timestamps = dset.split("_")[-1].replace(".nc","").split("-")
        t1, t2 = timestamps
        # Check if the timestamps are equal
        if t1 == t2:
            pass
        else:
            datasets.append(dset)
            
    # Next, determine if the dataset is either for the given instrument
    # or an ancillary instrument which supplies and input variable
    catalog = datasets
    datasets = []
    ancillary = []
    for dset in catalog:
        if re.search(stream, dset.split("/")[-1]) is None:
            ancillary.append(dset)
        else:
            datasets.append(dset)
            
    # Finally, check that deployment numbers match what is in deployments metadata
    catalog = datasets
    datasets = []
    for dset in catalog:
        dep = re.findall("deployment[\d]{4}", dset)[0]
        depNum = int(dep[-4:])
        if depNum not in list(deployments["deploymentNumber"]):
            pass
        else:
            datasets.append(dset)
            
    return datasets

In [60]:
for row in metadata.index:
    # Get the method and stream
    method, stream = metadata.loc[row,"method"], metadata.loc[row, "stream"]
    
    if "air" in stream:
        continue
    
   
    # Get the THREDDS url
    thredds_url = OOINet.get_thredds_url(refdes, method, stream, goldCopy=True)
    
    # Get the catalog
    catalog = OOINet.get_thredds_catalog(thredds_url)
    
    # Clean the catalog
    catalog = clean_catalog(catalog, stream, deployments)
    
    # Remove unwanted datasets from the catalog
    for dataset in catalog:
        if "blank" in dataset:
            catalog.remove(dataset)
    
    # Create a directory to save the data
    save_dir = f"../data/{refdes}/{method}/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    else:
        pass
    
    # Download the files to the save directory
    OOINet.download_netCDF_files(catalog, goldCopy=True, saveDir=save_dir)

[########################################] | 100% Completed |  4.0s
[########################################] | 100% Completed |  8.1s
[########################################] | 100% Completed |  1.0s


In [61]:
refdes, method, stream, save_dir

('GP03FLMB-RIM01-02-CTDMOG060',
 'telemetered',
 'ctdmo_ghqr_sio_mule_instrument',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/telemetered/')

In [62]:
# For the PCO2A, need to get just the "water" stream
os.listdir("..")

['.gitignore',
 'README.md',
 'scripts',
 'utils.py',
 'environment.yaml',
 '.ipynb_checkpoints',
 '.git',
 'documents',
 'bottle_utils.ipynb',
 'figures',
 'user_info.yaml',
 'bottle_utils.py',
 '__pycache__',
 'data']

In [63]:
save_dir

'../data/GP03FLMB-RIM01-02-CTDMOG060/telemetered/'

### Merge Datasets

With the datasets downloaded to a local directory, we now want to combine the datasets delivered via the different methods into a single dataset. This dataset should have the most complete data record available for the given reference designator.


#### Load the data
First, load the downloaded data into xarray datasets.

In [64]:
# List the telemetered data sets
telemetered_files = os.listdir(f"../data/{refdes}/telemetered")
telemetered_files = sorted([f"../data/{refdes}/telemetered/" + f for f in telemetered_files if "metbk" not in f])
telemetered_files

['../data/GP03FLMB-RIM01-02-CTDMOG060/telemetered/deployment0004_GP03FLMA-RIM01-02-CTDMOG040-telemetered-ctdmo_ghqr_sio_mule_instrument_20160701T040001-20160710T120001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/telemetered/deployment0006_GP03FLMA-RIM01-02-CTDMOG040-telemetered-ctdmo_ghqr_sio_mule_instrument_20180724T000001-20180816T080001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/telemetered/deployment0007_GP03FLMA-RIM01-02-CTDMOG040-telemetered-ctdmo_ghqr_sio_mule_instrument_20190929T080001-20200123T160001.nc']

In [65]:
# For the PCO2A, need just the "water" data stream
remove_files = []
for file in telemetered_files:
    if "air" in file:
        remove_files.append(file)

for f in remove_files:
    telemetered_files.remove(f)
    
telemetered_files

['../data/GP03FLMB-RIM01-02-CTDMOG060/telemetered/deployment0004_GP03FLMA-RIM01-02-CTDMOG040-telemetered-ctdmo_ghqr_sio_mule_instrument_20160701T040001-20160710T120001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/telemetered/deployment0006_GP03FLMA-RIM01-02-CTDMOG040-telemetered-ctdmo_ghqr_sio_mule_instrument_20180724T000001-20180816T080001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/telemetered/deployment0007_GP03FLMA-RIM01-02-CTDMOG040-telemetered-ctdmo_ghqr_sio_mule_instrument_20190929T080001-20200123T160001.nc']

In [66]:
# List the recovered_host data sets
recovered_host_files = os.listdir(f"../data/{refdes}/recovered_host")
recovered_host_files = sorted([f"../data/{refdes}/recovered_host/" + f for f in recovered_host_files if "metbk" not in f])
recovered_host_files

['../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_host/deployment0004_GP03FLMA-RIM01-02-CTDMOG040-recovered_host-ctdmo_ghqr_sio_mule_instrument_20160701T011501-20170717T200001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_host/deployment0005_GP03FLMA-RIM01-02-CTDMOG040-recovered_host-ctdmo_ghqr_sio_mule_instrument_20170713T230001-20180725T200001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_host/deployment0006_GP03FLMA-RIM01-02-CTDMOG040-recovered_host-ctdmo_ghqr_sio_mule_instrument_20180723T230001-20190928T200001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_host/deployment0007_GP03FLMA-RIM01-02-CTDMOG040-recovered_host-ctdmo_ghqr_sio_mule_instrument_20190929T200001-20210725T200001.nc']

In [67]:
# For the PCO2A, need just the "water" data stream
remove_files = []
for file in recovered_host_files:
    if "air" in file:
        remove_files.append(file)

for f in remove_files:
    recovered_host_files.remove(f)
    
recovered_host_files

['../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_host/deployment0004_GP03FLMA-RIM01-02-CTDMOG040-recovered_host-ctdmo_ghqr_sio_mule_instrument_20160701T011501-20170717T200001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_host/deployment0005_GP03FLMA-RIM01-02-CTDMOG040-recovered_host-ctdmo_ghqr_sio_mule_instrument_20170713T230001-20180725T200001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_host/deployment0006_GP03FLMA-RIM01-02-CTDMOG040-recovered_host-ctdmo_ghqr_sio_mule_instrument_20180723T230001-20190928T200001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_host/deployment0007_GP03FLMA-RIM01-02-CTDMOG040-recovered_host-ctdmo_ghqr_sio_mule_instrument_20190929T200001-20210725T200001.nc']

In [68]:
# List the recovered_inst data sets
recovered_inst_files = os.listdir(f"../data/{refdes}/recovered_inst")
recovered_inst_files = sorted([f"../data/{refdes}/recovered_inst/" + f for f in recovered_inst_files if "metbk" not in f])

recovered_inst_files

['../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_inst/deployment0001_GP03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20130721T224501-20140615T171501.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_inst/deployment0002_GP03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20140618T060001-20150605T144501.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_inst/deployment0003_GP03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20150606T224501-20160628T150001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_inst/deployment0004_GP03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20160701T011501-20170718T150001.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_inst/deployment0005_GP03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20170713T230001-20180726T154501.nc',
 '../data/GP03FLMB-RIM01-02-CTDMOG060/recovered_inst/deployment0006_GP03FLMA-RIM01-02-CTDMOG040-reco

Load the datasets:

In [69]:
from dask.diagnostics import ProgressBar

In [70]:
refdes

'GP03FLMB-RIM01-02-CTDMOG060'

In [71]:
def open_datasets(datasets, refdes):
    """Opens datasets saved locally into an xarray dataset."""
    
    OOINet.REFDES = refdes
    
    # Clean the catalog
    
    
    # Load the datasets into a concatenated xarray DataSet
    with ProgressBar():
        print("\n"+f"Loading netCDF_files for {OOINet.REFDES}:")
        ds = xr.open_mfdataset(datasets, preprocess=OOINet._preprocess, parallel=True)
        
    # Add in the English name of the dataset
    refdes = "-".join(ds.attrs["id"].split("-")[:4])
    vocab = OOINet.get_vocab(refdes)
    ds.attrs["Location_name"] = " ".join((vocab["tocL1"].iloc[0],
                                          vocab["tocL2"].iloc[0],
                                          vocab["tocL3"].iloc[0]))    

    return ds

In [72]:
refdes

'GP03FLMB-RIM01-02-CTDMOG060'

In [73]:
tele_data = open_datasets(telemetered_files, refdes)
host_data = open_datasets(recovered_host_files, refdes)
inst_data = open_datasets(recovered_inst_files, refdes)


Loading netCDF_files for GP03FLMB-RIM01-02-CTDMOG060:
[########################################] | 100% Completed |  2.2s

Loading netCDF_files for GP03FLMB-RIM01-02-CTDMOG060:
[########################################] | 100% Completed |  1.9s

Loading netCDF_files for GP03FLMB-RIM01-02-CTDMOG060:
[########################################] | 100% Completed |  4.0s


#### Optional Step: Process the dataset
An additional step is to process the datasets to clean up the datasets and get rid of 

In [74]:
sys.path.append("/home/areed/Documents/OOI/oceanobservatories/ooi-data-explorations/python/")

In [35]:
from ooi_data_explorations.uncabled import process_ctdbp

In [36]:
inst_data = process_ctdbp.ctdbp_instrument(inst_data)
host_data = process_ctdbp.ctdbp_datalogger(host_data)
tele_data = process_ctdbp.ctdbp_datalogger(tele_data)

ValueError: One or more of the specified variables cannot be found in this dataset

#### Merge data
Now, we need to merge the data. First, we iterate through the data variables for each dataset, identify any which are unique to a given dataset, and broadcast them to the other datasets. This step is necessary to allow the datasets to combine. Once each dataset has the same data variables, we utilize ```xr.combine_first``` to combine the datasets. We assume that the instrument record, if available, is the best and most complete dataset and utilize the telemetered and recovered_host datasets to fill in the gaps.

In [75]:
# Need to make sure each dataset has the same variables
for var in tele_data.variables:
    if var not in host_data.variables:
        host_data[var] = tele_data[var].broadcast_like(host_data["time"])
        
for var in host_data.variables:
    if var not in tele_data.variables:
        tele_data[var] = host_data[var].broadcast_like(tele_data["time"])

In [76]:
# Merge the telemetered dataset and host_dataset
tele_host = tele_data.combine_first(host_data)
data = tele_host

In [77]:
for var in tele_host.variables:
    if var not in inst_data.variables:
        inst_data[var] = tele_host[var].broadcast_like(inst_data["time"])

for var in inst_data.variables:
    if var not in tele_host.variables:
        tele_host[var] = inst_data[var].broadcast_like(tele_host["time"])

In [78]:
# Merge the instrument dataset with the combined telemetered-recovered_host dataset
data = inst_data.combine_first(tele_host)

#### Save the results
With the merged datasets, we can save the results locally as a netCDF file. However, some data variables contain improperly formatted datetimes and timestamps which will cause an error when saving. Generally, these data variables do not contain particularly useful information for a science-user and can be dropped before saving.

In [79]:
data

<xarray.Dataset>
Dimensions:                                      (time: 279051)
Coordinates:
  * time                                         (time) datetime64[ns] 2013-0...
Data variables: (12/32)
    practical_salinity                           (time) float64 dask.array<chunksize=(31339,), meta=np.ndarray>
    ctdmo_seawater_conductivity_qartod_executed  (time) object dask.array<chunksize=(31339,), meta=np.ndarray>
    ctdmo_seawater_conductivity_qartod_results   (time) float64 dask.array<chunksize=(31339,), meta=np.ndarray>
    ctd_time                                     (time) datetime64[ns] dask.array<chunksize=(31339,), meta=np.ndarray>
    density_qc_executed                          (time) float64 dask.array<chunksize=(31339,), meta=np.ndarray>
    conductivity                                 (time) float64 dask.array<chunksize=(31339,), meta=np.ndarray>
    ...                                           ...
    practical_salinity_qc_executed               (time) float64 dask.array<chunksize=(31339,), meta=np.ndarray>
    depth                                        (time) float64 dask.array<chunksize=(31339,), meta=np.ndarray>
    inductive_id                                 (time) float32 dask.array<chunksize=(31339,), meta=np.ndarray>
    density_qc_results                           (time) float64 dask.array<chunksize=(31339,), meta=np.ndarray>
    ctdmo_seawater_pressure_qartod_executed      (time) object dask.array<chunksize=(31339,), meta=np.ndarray>
    sio_controller_timestamp                     (time) datetime64[ns] dask.array<chunksize=(138015,), meta=np.ndarray>
Attributes: (12/70)
    node:                               RIM01
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdmo_ghqr_instrument_recovered
    ...                                 ...
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                49.9795
    lon:                                -144.254
    Location_name:                      Global Station Papa Flanking Subsurfa...

In [80]:
refdes

'GP03FLMB-RIM01-02-CTDMOG060'

Save the data as a netCDF file using h5netcdf compression

In [81]:
data.to_netcdf(f"../data/{refdes}_combined.nc", engine="h5netcdf")

Close the dataset so it can be operated on

In [82]:
data.close()

In [83]:
os.listdir("../data/")

['GP03FLMA-RIM01-02-CTDMOG040_combined.nc',
 'GI03FLMA-RIS01-04-PHSENF000_annotations.csv',
 'GP03FLMB-RIS01-04-PHSENF000_deployments.csv',
 'GP03FLMA-RIS01-04-PHSENF000_deployments.csv',
 'GP03FLMA-RIS01-05-FLORTD000',
 'CP03ISSM-SBD12-04-PCO2AA000_deployments.csv',
 'bottle_data',
 'GP03FLMA-RIM01-02-CTDMOG040_annotations.csv',
 'CP03ISSM-SBD12-04-PCO2AA000',
 'GP03FLMB-RIS01-03-DOSTAD000_deployments.csv',
 'GP03FLMA-RIS01-04-PHSENF000',
 'GI01SUMO-SBD12-04-PCO2AA000_combined.nc',
 'ooi-citations',
 'GI01SUMO-RID16-05-PCO2WB000_combined.nc',
 'GI03FLMA-RIS01-04-PHSENF000_combined.nc',
 'GP03FLMA-RIM01-02-CTDMOG040_deployments.csv',
 '.ipynb_checkpoints',
 'GP03FLMB-RIS01-03-DOSTAD000_combined.nc',
 'GP03FLMB-RIS01-04-PHSENF000_combined.nc',
 'GP03FLMA-RIS01-03-DOSTAD000_annotations.csv',
 'CP01CNSM-MFD35-05-PCO2WB000_deployments.csv',
 'GP03FLMA-RIS01-03-DOSTAD000',
 'CP01CNSM-MFD35-05-PCO2WB000',
 'CP01CNSM-MFD35-05-PCO2WB000_combined.nc',
 'GP03FLMB-RIS01-03-DOSTAD000',
 'CP03ISSM-

---
## Annotations
Annotations contain important qualitative assessments of data quality from the instrument operators. They may range from explanations for why data is missing for a given time period to information about biofouling or other data quality issues. Annotations can be downloaded from OOINet for a particular reference designator.

In [84]:
# Download the annotations for each reference designator
annotations = OOINet.get_annotations(refdes)
annotations

,@class,id,subsite,node,sensor,method,stream,beginDT,endDT,annotation,exclusionFlag,source,qcFlag,parameters
0,.AnnotationRecord,154,GP03FLMB,None,None,None,None,1374647520000,1403049600000,Deployment 1: Mooring was deployed 5-10m shall...,False,lgarzio@marine.rutgers.edu,0,[]
1,.AnnotationRecord,156,GP03FLMB,None,None,None,None,1433798700000,1467570900000,Deployment 3: Mooring was deployed 10m deeper ...,False,lgarzio@marine.rutgers.edu,0,[]
2,.AnnotationRecord,157,GP03FLMB,None,None,None,None,1467673620000,1500303720000,Deployment 4: Mooring was deployed 10m deeper ...,False,lgarzio@marine.rutgers.edu,0,[]
3,.AnnotationRecord,155,GP03FLMB,None,None,None,None,1403229600000,1433635200000,Deployment 2: Mooring was deployed 10-15m shal...,False,lgarzio@marine.rutgers.edu,0,[]
4,.AnnotationRecord,142,GP03FLMB,None,None,None,None,1500072540000,1532538660000,Deployment 5: Telemetered data are not availab...,False,lgarzio@marine.rutgers.edu,0,[]
5,.AnnotationRecord,2165,GP03FLMB,RIM01,02-CTDMOG060,None,None,1500072540000,1532538660000,Deployment 5: It was discovered upon recovery ...,False,cdobson@whoi.edu,9,[]


Save the annotations to local directory

In [85]:
annotations.to_csv(f"../data/{refdes}_annotations.csv")

---
## Data Explorer
---
The data from Data Explorer are hosted via ERDDAP. To interact with Data Explorer's ERDDAP, we'll utilize the python package ```erddapy```.

In [ ]:
from erddapy import ERDDAP

**Data Explorer ERDDAP url**

In [ ]:
dataExplorer = "http://erddap.dataexplorer.oceanobservatories.org/erddap"

Connect to the Data Explorer ERDDAP

In [ ]:
erd = ERDDAP(server=dataExplorer)

Search for ```PHSEN``` on the Irminger Array

In [ ]:
search_url = erd.get_search_url(search_for="gi03flma phsen", 
                                protocol="tabledap",
                                response="csv")

Get the dataset ids for the available PHSEN datasets on the Irminger Sea Flanking Mooring A

In [ ]:
dataset_ids = pd.read_csv(search_url)["Dataset ID"]
dataset_ids

Download the dataset from ERDDAP

In [ ]:
# Select the dataset id of the instrument you want to download
dataset_id = "ooi-gi03flma-ris01-04-phsenf000"

# Get the download url
download_url = erd.get_download_url(dataset_id=dataset_id, 
                                    protocol="tabledap",
                                    response="opendap")

# Set up the parameters for the dataset request from the ERDDAP server
erd.dataset_id = dataset_id
erd.response = "nc"
erd.protocol = "tabledap"

Open the requested dataset using ```xarray```

In [ ]:
ds = erd.to_xarray()
ds = ds.swap_dims({"obs":"time"})
ds = ds.sortby("time")
ds